In [ ]:
import surprise
import pandas as pd
import numpy as np
import urllib

In [ ]:
df = ...

In [ ]:
lower_rating = dataset['rating'].min()
upper_rating = dataset['rating'].max()

In [ ]:
reader = surprise.Reader(rating_scale=(0.5,4.))
data = surprised.Dataset.load_from_df(dataset,reader)

In [ ]:
alg = surprise.SVDpp()
# SVD ++ (from netflix challenge) is a popular method for fitting recommender systems 
# extends vanilla SVD alg by only optimising known terms and performing regularisation
output = alg.fit(data.build_full_trainset())
pred = alg.predict(uid='50',iid='52')
score=pred.est
print(score)

In [ ]:
# making recommendations
# find movie ids that user 50 didnt rate - dont want to recommend a movie they already watched
iids = dataset['iid'].unique()
iids50 = dataset.loc(dataset['uid']==50,'iid')
iids_to_pred = np.setdiffid(iids,iids50)

In [ ]:
# predict score of movie ids that user 50 didnt rate and find best one
testset = [[50,iid,4.] for iid in iids_to_pred]
predictions=alg.test(testset)
predictions[0]
# each pred is a special obj - in order to find the best, convert obj into array of pred ratings
pred_ratings = np.array([pred.est for pred in predictions])
i_max = pred_ratings.argmax() # find index of the max predicted rating
iid = iids_to_pred[i_max] #use this to find corres iid to recommend

#can get top n items for user 50 -- replace argmax() with argpartition

In [ ]:
# tuning and evaling model
# can tune dimension DD affecting size of UU and VV, learning rate,
# regularization term, number of epochs

param_grid = {
    'lr_all': [.001,.01],
    'reg_all': [.1,.5]
}
gs = surprise.model_selection.GridSearchCV(surprise.SVDpp, param_grid, measures=['rmse','mae'], cv=3)
gs.fit(data)
print(gs.best_params['rmse'])

In [ ]:
alg = surprise.SVDpp(lr_all=0.001)
output = surprise.model_selection.cross_validate(alg, data, verbose=True)